In [1]:
import pandas as pd

In [4]:
def create_bmom(etf_key, etf_path, weights):
    etf_data = pd.read_csv(etf_path)

    data = etf_data[['Date', etf_key]].copy()
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m%d', errors='coerce')
    data[etf_key] = pd.to_numeric(data[etf_key], errors='coerce')
    data = data.dropna().sort_values('Date').set_index('Date')

    px = data[etf_key]  
    momentums = {}
    weighted_momentums = {}
    for lookback, weight in weights.items():
        momentums[lookback] = px.pct_change(lookback)
        weighted_momentums[lookback] = momentums[lookback] * weight

    # Concatenate weighted momentums
    wm_df = pd.concat(weighted_momentums, axis=1)

    # Find first index where ALL lookbacks are valid
    first_valid = wm_df.dropna().index[0]

    # Sum only from that index onward
    bmom = wm_df.loc[first_valid:].sum(axis=1)
    daily_returns = px.pct_change(1)

    dates = bmom.index  # aligned dates for bmom

    return bmom, momentums, dates, daily_returns


bmom, momentums, dates, daily_returns = create_bmom(
    'H5A4',
    'Data Files\H5A4.csv',
    {20: 0.15, 60: 0.35, 120: 0.35, 240: 0.15}
)


print(bmom)



Date
1997-12-05    0.000527
1997-12-08    0.000294
1997-12-09    0.000037
1997-12-10    0.000962
1997-12-11    0.002043
                ...   
2025-11-05    0.010972
2025-11-06    0.010983
2025-11-07    0.009400
2025-11-10    0.011850
2025-11-11    0.011254
Length: 7300, dtype: float64


<>:33: SyntaxWarning: invalid escape sequence '\H'
<>:33: SyntaxWarning: invalid escape sequence '\H'
C:\Users\nimas\AppData\Local\Temp\ipykernel_48804\3018610235.py:33: SyntaxWarning: invalid escape sequence '\H'
  'Data Files\H5A4.csv',
